---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [37]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
# % matplotlib inline

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [41]:
def graph_identification():
    
    for j in range(len(P1_Graphs)):
#         plt.subplot(3,3,j+1)
        G = P1_Graphs[j]
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values]

#         plt.plot(degree_values, histogram,'o')
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of nodes')
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.show()

    # GRAPH 0 AND 3 IS GENERATED BY Preferential Attachment ('PA')
    swl = []
    for j in [1, 2, 4]:
        G = P1_Graphs[j]
        if nx.average_clustering(G) > 0.1:
            swl.append(j)
    # GRAPH 1 AND 2 IS GENERATED BY Small World with low probability of rewiring ('SW_L')
    # GRAPH 4 IS GENERATED BY High World with low probability of rewiring ('SW_H')
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [31]:
G = nx.read_gpickle('email_prediction.txt')


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [68]:
def salary_predictions():
    
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    
    # DataFrame setup
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['Degree'] = pd.Series(G.degree())
    df['Closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['PR'] = pd.Series(nx.pagerank(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
    
    #Predicting
    Train_df = df[~pd.isnull(df['ManagementSalary'])]
    Test_df = df[pd.isnull(df['ManagementSalary'])]
    features =['Clustering', 'Betweeness', 'Degree',
           'Closeness', 'PR', 'degree_centrality', 'closeness_centrality']
    X_train = Train_df[features]
    y_train = Train_df['ManagementSalary']
    X_test = Test_df[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scaled, y_train)
    predictions = (clf.predict_proba(X_test_scaled)[:,1]).tolist()
    return pd.Series(predictions,X_test.index)

salary_predictions()

1       0.016228
2       0.930000
5       1.000000
8       0.309424
14      0.036547
18      0.117443
27      0.062402
30      0.817143
31      0.367361
34      0.060000
37      0.020000
40      0.134173
45      0.020851
54      0.323044
55      0.530241
60      0.211002
62      1.000000
65      0.960000
77      0.051165
79      0.095565
97      0.010000
101     0.000000
103     0.456991
108     0.034227
113     0.100000
122     0.010000
141     0.268445
142     1.000000
144     0.000323
145     0.484155
          ...   
913     0.000000
914     0.000000
915     0.000000
918     0.010541
923     0.000000
926     0.025154
931     0.000323
934     0.000000
939     0.000000
944     0.000000
945     0.000000
947     0.010357
950     0.024244
951     0.000000
953     0.000323
959     0.000000
962     0.000000
963     0.148022
968     0.000357
969     0.002157
974     0.003874
984     0.000000
987     0.011969
989     0.000357
991     0.000357
992     0.000000
994     0.000000
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [63]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})


,Future Connection,Preferential Attachment,Common Neighbours,Jaccard Coefficient,Resource Allocation,Adamic Adar,cn_soundarajan_hopcroft,ra_soundarajan_hopcroft
"(6, 840)",0.0,2070,9,0.073770,0.136721,2.110314,9,0.0
"(4, 197)",0.0,3552,2,0.015504,0.008437,0.363528,2,0.0
"(620, 979)",0.0,28,0,0.000000,0.000000,0.000000,0,0.0
"(519, 872)",0.0,299,2,0.060606,0.039726,0.507553,2,0.0
"(382, 423)",0.0,205,0,0.000000,0.000000,0.000000,0,0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [67]:
def new_connections_predictions():
        
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    
    #DataFrame setup
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbours'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in future_connections.index]
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    # community assignment
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]   
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    
    #Predictions
    Train_df = future_connections[~pd.isnull(future_connections['Future Connection'])]
    Test_df = future_connections[pd.isnull(future_connections['Future Connection'])]
    features =['Preferential Attachment', 'Common Neighbours',
       'Jaccard Coefficient', 'Resource Allocation', 'Adamic Adar',
       'cn_soundarajan_hopcroft', 'ra_soundarajan_hopcroft']
    X_train = Train_df[features]
    y_train = Train_df['Future Connection']
    X_test = Test_df[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scaled, y_train)
    predictions = (clf.predict_proba(X_test_scaled)[:,1]).tolist()
    return pd.Series(predictions,X_test.index)

new_connections_predictions()

(107, 348)    0.027101
(542, 751)    0.012506
(20, 426)     0.577256
(50, 989)     0.012506
(942, 986)    0.012840
(324, 857)    0.012593
(13, 710)     0.131673
(19, 271)     0.147741
(319, 878)    0.012456
(659, 707)    0.012533
(49, 843)     0.012605
(208, 893)    0.012392
(377, 469)    0.014032
(405, 999)    0.016320
(129, 740)    0.012518
(292, 618)    0.018848
(239, 689)    0.012371
(359, 373)    0.016191
(53, 523)     0.419443
(276, 984)    0.012333
(202, 997)    0.012554
(604, 619)    0.032482
(270, 911)    0.012456
(261, 481)    0.060907
(200, 450)    0.987796
(213, 634)    0.012474
(644, 735)    0.032666
(346, 553)    0.012414
(521, 738)    0.011816
(422, 953)    0.014594
                ...   
(672, 848)    0.012456
(28, 127)     0.990988
(202, 661)    0.012414
(54, 195)     0.999994
(295, 864)    0.012523
(814, 936)    0.012419
(839, 874)    0.012840
(139, 843)    0.012593
(461, 544)    0.013818
(68, 487)     0.013216
(622, 932)    0.012615
(504, 936)    0.017355
(479, 528) 